In [1]:
# Importing necessary models
import warnings
warnings.filterwarnings('ignore')

import smtplib
import pandas as pd
import numpy as np
import datetime as dt
import pandas.stats.moments as st
import sqlite3 as sql
import time
%matplotlib inline
import os
main_dir = os.getcwd()

from option_slam_earnings_db import *
os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\DataCollection')

from yahoo_query import *
from reuters_query import reuters_query

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Trading\\Modules\\Earnings')
from yahoo_earnings import *

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\Stock Universe')
cad_stocks = pd.read_csv('cad_names.csv')['Symbol'].tolist()
usd_stocks = pd.read_csv('us_names.csv')['Symbol'].tolist()

os.chdir('C:\\Users\\Fang\\Desktop\\Python Trading\\Trading\\Data\\DBs')